# 📥📤Inputs and Outputs (Model I/O)

In LangChain, Model I/O (Input/Output) is the layer that defines how you interact with models. It provides standardized abstractions for sending prompts into models and parsing their responses.

## 🔑 Why do we need Model I/O
* **Consistency across models**: You can swap between OpenAI, Anthropic, Llama, etc. without changing your business logic, since LangChain standardizes the I/O.
* **Structured outputs**: Instead of working with free-form text, you can parse model responses into JSON, Pydantic objects, or enums. This reduces errors and hallucinations.
* **Composable pipelines**: Model I/O integrates with other LangChain pieces (retrievers, memory, tools) so you can **chain models and functions easily**.
* **Flexibility**:
    * Fine control over prompts (system + human messages, variable substitution).
    * Choice of parsers (regex, structured, function-calling).
    * Input/output transformations before/after the model.

# 📥Inputs
Inputs are the starting point of any interaction with an LLM-powered workflow. They represent the information you feed into your chain, agent, or model so it can generate a useful output.

**Types of Inputs**

* Raw Strings
  > The simplest input is just plain text (e.g., "What’s the capital of France?").
* Structured Inputs (Dictionaries)
  > Many LangChain components expect a dictionary with key–value pairs.
  
Inputs are usually combined with **PromptTemplates and variables**.

# 📝 Prompt Templates and Input Variables
A Prompt template in LangChain is a reusable way to create dynamic prompts for LLMs.
Instead of hardcoding the full prompt string, you define a template with placeholders, and later fill those placeholders with variables at runtime.
This ensures:

* Consistency – Reuse the same structure for different inputs.
* Flexibility – Easily swap variables without rewriting prompts.
* Maintainability – Keeps your LLM app organized as prompts grow.

LangChain Uses two types of prompt templates
1. **PromptTemplate** (for completion models)
2. **ChatPromptTemplate** (for chat models)

## ✨PromptTemplate
* **Purpose**: Used for text-based LLMs (completion models).
* **Output**: A single string prompt.
* **Use case**: When you want to generate a plain text prompt (no roles like “system” or “user”).

In [ ]:
## google colab
# %pip install langchain openai
# %pip install langchain langchain-community openai
# %pip install langchain_openai

In [17]:
import os
from dotenv import load_dotenv
from langchain.prompts import PromptTemplate, ChatPromptTemplate
from langchain_openai import OpenAI

In [18]:
load_dotenv()
# os.environ["OPENAI_API_KEY"] = "<Your API KEY>" # Google colab

In [19]:
llm = OpenAI(model="gpt-3.5-turbo-instruct", temperature=0, api_key=os.getenv("OPEN_API_KEY"))

In [20]:
from langchain.prompts import PromptTemplate

# Single input variable: "product"
single_input_template = PromptTemplate(
    input_variables=["product"],
    template="What are three benefits of using {product}?"
)

# template = "What are three benefits of using {product}?"
# prompt = PromptTemplate.from_template(template)
# print(prompt.format(product="LangChain"))

# Format with input
formatted_prompt = single_input_template.format(product="LangChain")
print(formatted_prompt)

What are three benefits of using LangChain?


In [22]:
response = llm.invoke(formatted_prompt)
print("LLM response:", response)

LLM response: 

1. Increased Efficiency: LangChain utilizes blockchain technology to automate and streamline language translation processes, resulting in increased efficiency and reduced turnaround times. This can save businesses time and resources, allowing them to focus on other important tasks.

2. Enhanced Security: With its decentralized and immutable nature, LangChain offers enhanced security for language translation services. This means that translations are less susceptible to tampering or manipulation, ensuring the accuracy and integrity of the translated content.

3. Cost Savings: By eliminating intermediaries and automating processes, LangChain can significantly reduce the costs associated with language translation services. This makes it a more affordable option for businesses and individuals looking to translate content into multiple languages.


## 💬ChatPromptTemplate
* **Purpose**: Used for chat-based LLMs (like OpenAI’s GPT-4, GPT-4o, GPT-3.5).
* **Output**: A list of message objects (with roles: system, human, ai, etc.).
* **Use case**: When you need structured multi-role prompts (system instructions + user queries).

In [ ]:
# Document as a string
document_text = """
LangChain is a framework for developing applications powered by language models.
It provides abstractions for prompts, chains, agents, memory, and integrates with many external tools.
"""

# Define the chat prompt template with system + human messages
chat_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant. Use the provided document to answer questions."),
    ("human", "Here is the document:\n{document}\n\nQuestion: {question}")
])

# Format the prompt with input variables
messages = chat_prompt.format_messages(
    document=document_text,
    question="What does LangChain provide?"
)

# Initialize the model
chat = ChatOpenAI(model="gpt-4o-mini", temperature=0, api_key=os.getenv("OPEN_API_KEY"))

# Get the response
response = chat.invoke(messages)
print(response.content)


In [ ]:
from langchain.prompts import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate
)

In [ ]:
# Document string
document_text = """
LangChain is a framework for developing applications powered by language models.
It provides abstractions for prompts, chains, agents, memory, and integrates with many external tools.
"""

# Create system and human message prompt templates
system_prompt = SystemMessagePromptTemplate.from_template(
    "You are a helpful assistant. Use the provided document to answer questions."
)

human_prompt = HumanMessagePromptTemplate.from_template(
    "Here is the document:\n{document}\n\nQuestion: {question}"
)

# Build the chat prompt template
chat_prompt = ChatPromptTemplate.from_messages([system_prompt, human_prompt])

# Format with inputs
messages = chat_prompt.format_messages(
    document=document_text,
    question="What does LangChain provide?"
)

# Initialize LLM
chat = ChatOpenAI(model="gpt-4o-mini", temperature=0,  api_key=os.getenv("OPEN_API_KEY"))

# Invoke the model
response = chat.invoke(messages)
print(response.content)

# Few Shots Prompt-Template

In LangChain, a few-shot template is a way to guide a language model by giving it examples of how it should respond before asking the real question. The idea comes from few-shot learning, where instead of training the model with lots of data, you show it just a handful of demonstrations right inside the prompt.

LangChain makes this easier with **FewShotPromptTemplate**. Instead of manually writing out all examples in one long string, you define a template for the examples and then let LangChain format them dynamically.

In [33]:
from langchain.prompts import (
    FewShotPromptTemplate, 
    FewShotChatMessagePromptTemplate, 
    PromptTemplate, 
    ChatPromptTemplate, 
    HumanMessagePromptTemplate, 
    AIMessagePromptTemplate
)

from langchain_openai import ChatOpenAI

In [53]:
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0, api_key=os.getenv("OPEN_API_KEY"))

In [49]:
# Template for each example
example_prompt = PromptTemplate(
    input_variables=["review", "sentiment"],
    template="Review: {review}\nSentiment: {sentiment}\n"
)

# Few-shot examples
examples = [
    {"review": "I loved the storyline and the acting was brilliant!", "sentiment": "Positive"},
    {"review": "The movie was too long and really boring.", "sentiment": "Negative"},
    {"review": "Amazing visuals and great soundtrack.", "sentiment": "Positive"},
]

# FewShotPromptTemplate with suffix for new input
few_shot_prompt = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    suffix="Review: {input}\nSentiment:",
    input_variables=["input"]
)

# Format with a new review
print(few_shot_prompt.format(input="The plot was confusing and poorly executed."))
# 4. Instantiate an LLM


# 5. Format the prompt with a new review
formatted_prompt = few_shot_prompt.format(input="The plot was confusing and poorly executed.")
print("=== Final Prompt Sent to LLM ===")
print(formatted_prompt)

# 6. Call the LLM
response = llm.invoke(formatted_prompt)
print("\n=== Model Output ===")
print(response.content)


Review: I loved the storyline and the acting was brilliant!
Sentiment: Positive


Review: The movie was too long and really boring.
Sentiment: Negative


Review: Amazing visuals and great soundtrack.
Sentiment: Positive


Review: The plot was confusing and poorly executed.
Sentiment:
=== Final Prompt Sent to LLM ===
Review: I loved the storyline and the acting was brilliant!
Sentiment: Positive


Review: The movie was too long and really boring.
Sentiment: Negative


Review: Amazing visuals and great soundtrack.
Sentiment: Positive


Review: The plot was confusing and poorly executed.
Sentiment:

=== Model Output ===
Negative


In [50]:
# 1) Build a chat prompt for each example (human → ai)
example_prompt = ChatPromptTemplate.from_messages([
    ("human", "Review: {review}"),
    ("ai", "{sentiment}"),
])

# 2) Few-shot examples
chat_examples = [
    {"review": "I loved the storyline and the acting was brilliant!", "sentiment": "Positive"},
    {"review": "The movie was too long and really boring.", "sentiment": "Negative"},
    {"review": "Amazing visuals and great soundtrack.", "sentiment": "Positive"},
]

# 3) Few-shot block (no input_variables here)
few_shot_prompt = FewShotChatMessagePromptTemplate(
    examples=chat_examples,
    example_prompt=example_prompt,
)

# 4) Full chat prompt (system + few-shot + new human input)
chat_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant that classifies movie reviews as Positive or Negative."),
    few_shot_prompt,
    ("human", "Review: {input}"),
])

# 6) Format and invoke
messages = chat_prompt.format_messages(input="The plot was confusing and poorly executed.")
print("=== Final Prompt Messages ===")
for m in messages:
    print(m.type.upper() + ":", m.content)

resp = llm.invoke(messages)
print("\n=== Model Output ===")
print(resp.content)


=== Final Prompt Messages ===
SYSTEM: You are a helpful assistant that classifies movie reviews as Positive or Negative.
HUMAN: Review: I loved the storyline and the acting was brilliant!
AI: Positive
HUMAN: Review: The movie was too long and really boring.
AI: Negative
HUMAN: Review: Amazing visuals and great soundtrack.
AI: Positive
HUMAN: Review: The plot was confusing and poorly executed.

=== Model Output ===
Negative


## Prompt Serialization
In LangChain, prompt serialization means saving your PromptTemplate (or ChatPromptTemplate) into a file (JSON/YAML) and later loading it back.
This makes your prompts:
* Reusable → Share prompts between projects or teams.
* Maintainable → Keep prompt logic outside your Python code.
* Configurable → Update prompts without touching source code.
  

In [55]:
from langchain.prompts import PromptTemplate, load_prompt

# Define a prompt
prompt = PromptTemplate(
    input_variables=["product"],
    template="What are three benefits of using {product}?"
)

# Save to JSON file
prompt.save("prompt.json")

# Load it back
loaded_prompt = load_prompt("prompt.json")

print("Loaded Prompt:", loaded_prompt.format(product="LangChain"))


Loaded Prompt: What are three benefits of using LangChain?


# 📤 Outputs
Just like Inputs are how you feed data into LangChain, Outputs are what you get back from a model, chain, or agent after processing.

### **Types of Outputs**
* **Raw Strings**
> The simplest form: just text from the LLM.
> Example: "The capital of France is Paris."

* **Message Objects**

> For chat models, outputs can be structured as AIMessage.

* **Structured Objects**
> Sometimes you want structured data, not free text.
> LangChain supports Output Parsers that turn text into:

    > 1. JSON / Python dicts
    > 2. Lists
    > 3. Numbers
    > 4. Pydantic models (typed objects)

#### 1️⃣ JSON / Python dicts

Use StructuredOutputParser or JsonOutputParser to enforce JSON.

In [51]:
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

# ---- JSON shape hint (plain text, no Pydantic) ----
schema_hint = """
Return ONLY one JSON object with this shape:
{
  "genre": string,
  "recommendations": [
    {
      "title": string,
      "year": integer,
      "reason": string,
      "director": string|null
    }
  ]
}
"""

parser = JsonOutputParser()

prompt = ChatPromptTemplate.from_messages([
    ("system",
     "You recommend movies strictly for the requested genre. "
     "Do not include explanations, code fences, or extra text. "
     "Output must be a single valid JSON object."),
    ("human",
     "Genre: {genre}\n"
     "How many: {k}\n"
     "Constraints: {constraints}\n"
     "Return exactly {k} items in 'recommendations'.\n\n"
     "{schema_hint}\n"
     "{format_instructions}")
]).partial(
    schema_hint=schema_hint,
    format_instructions=parser.get_format_instructions()
)

chain = prompt | llm | parser

out = chain.invoke({
    "genre": "Science Fiction",
    "k": 5,
    "constraints": "Prefer post-2010; include at least one non-English film."
})

print(out)  # -> plain dict


{'genre': 'Science Fiction', 'recommendations': [{'title': 'Arrival', 'year': 2016, 'reason': 'A thought-provoking exploration of language and time.', 'director': 'Denis Villeneuve'}, {'title': 'Ex Machina', 'year': 2014, 'reason': 'A tense examination of artificial intelligence and ethics.', 'director': 'Alex Garland'}, {'title': 'Blade Runner 2049', 'year': 2017, 'reason': "A visually stunning sequel that expands on the original's themes.", 'director': 'Denis Villeneuve'}, {'title': 'Annihilation', 'year': 2018, 'reason': 'A surreal journey into an alien environment with deep psychological themes.', 'director': 'Alex Garland'}, {'title': 'The Platform', 'year': 2019, 'reason': 'A Spanish film that offers a unique take on class and survival.', 'director': 'Galder Gaztelu-Urrutia'}]}


#### 2️⃣ Lists
Use `CommaSeparatedListOutputParser` for comma-separated outputs.

In [42]:
from langchain.output_parsers import CommaSeparatedListOutputParser

parser = CommaSeparatedListOutputParser()
print(parser.parse("apple, banana, cherry"))
# → ["apple", "banana", "cherry"]


['apple', 'banana', 'cherry']


#### 3️⃣ Numbers
Use a simple OutputParser that ensures numeric type.

In [43]:
from langchain.schema import BaseOutputParser

class IntOutputParser(BaseOutputParser[int]):
    def parse(self, text: str) -> int:
        return int(text.strip())

parser = IntOutputParser()
print(parser.parse(" 42 "))
# → 42


42


#### 4️⃣ Pydantic Models (typed objects)

With PydanticOutputParser, you define a schema → enforce typed outputs.

In [54]:
from typing import Optional, List
from pydantic import BaseModel, Field
from langchain_core.output_parsers import PydanticOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

# 1) Define your typed schema
class Recommendation(BaseModel):
    title: str = Field(..., description="Movie title")
    year: int = Field(..., description="Release year")
    reason: str = Field(..., description="Why this matches the request")
    director: Optional[str] = Field(None, description="Director if known")

class MovieRecs(BaseModel):
    genre: str
    recommendations: List[Recommendation]

# 2) Make a Pydantic parser for that schema
parser = PydanticOutputParser(pydantic_object=MovieRecs)

# 3) Build the prompt, using the parser’s format instructions
prompt = ChatPromptTemplate.from_messages([
    ("system",
     "You recommend movies strictly for the requested genre. "
     "Do not include explanations, code fences, or extra text. "
     "Output must be a single valid JSON object that matches the schema."),
    ("human",
     "Genre: {genre}\n"
     "How many: {k}\n"
     "Constraints: {constraints}\n"
     "Return exactly {k} items in 'recommendations'.\n\n"
     "{format_instructions}")
]).partial(
    format_instructions=parser.get_format_instructions()
)

chain = prompt | llm | parser

# 5) Invoke → returns a MovieRecs Pydantic object
out: MovieRecs = chain.invoke({
    "genre": "Science Fiction",
    "k": 5,
    "constraints": "Prefer post-2010; include at least one non-English film."
})

# Pydantic object usage
print(out)                 # -> MovieRecs(...)
print(out.dict())          # -> plain dict
print(out.recommendations) # -> List[Recommendation]


genre='Science Fiction' recommendations=[Recommendation(title='Arrival', year=2016, reason='A thought-provoking exploration of language and time through the arrival of extraterrestrial beings.', director='Denis Villeneuve'), Recommendation(title='Ex Machina', year=2014, reason='A gripping tale of artificial intelligence and the ethical dilemmas surrounding it.', director='Alex Garland'), Recommendation(title='Blade Runner 2049', year=2017, reason='A visually stunning sequel that expands on the themes of identity and humanity.', director='Denis Villeneuve'), Recommendation(title='Annihilation', year=2018, reason='A mind-bending journey into a mysterious zone that alters everything it touches.', director='Alex Garland'), Recommendation(title='The Platform', year=2019, reason='A Spanish film that uses a unique setting to explore themes of class and survival.', director='Galder Gaztelu-Urrutia')]
{'genre': 'Science Fiction', 'recommendations': [{'title': 'Arrival', 'year': 2016, 'reason': 

C:\Users\Dagi\AppData\Local\Temp\ipykernel_49416\2829697271.py:48: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  print(out.dict())          # -> plain dict
